In [12]:
import pyspark.pandas as ps 
from pyspark.sql import SparkSession
import pandas as pd 
import pyspark.sql.functions as f 

In [4]:
spark = SparkSession.builder.config("spark.executor.memory","10g").config("spark.driver.memory","2G").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/11/30 11:46:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/11/30 11:46:11 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [124]:
data = spark.read.csv("/home/himedia/data/부동산실거래가1998-2023.csv",header=True)

In [10]:
data.show()

+---------------------------------+-----------+------------+--------+--------------+---+--------+
|                           시군구|     단지명|전용면적(㎡)|계약년월|거래금액(만원)| 층|건축년도|
+---------------------------------+-----------+------------+--------+--------------+---+--------+
|강원특별자치도 양구군 양구읍 상리|       경림|        59.4|  200509|          5000| 11|    1998|
|           경기도 의정부시 민락동| 한라비발디|       84.99|  200510|         12750| 19|    2003|
|           경기도 의정부시 신곡동|      신일1|      59.878|  200510|          5750|  3|    1997|
|           경기도 의정부시 신곡동|       풍림|       49.83|  200510|          5500|  8|    1998|
|           경기도 의정부시 용현동|용현현대1차|      129.73|  200511|         12200| 11|    1992|
|           경기도 의정부시 용현동|용현현대1차|      129.73|  200511|         11500| 17|    1992|
|     강원특별자치도 강릉시 견소동|   송정한신|       43.38|  200601|          4200|  1|    1997|
|       강원특별자치도 강릉시 교동|  교동1주공|       59.89|  200601|          7500|  2|    1999|
|       강원특별자치도 강릉시 교동|  교동1주공|       59.89|  200601|          8500

In [11]:
data.printSchema()

root
 |-- 시군구: string (nullable = true)
 |-- 단지명: string (nullable = true)
 |-- 전용면적(㎡): string (nullable = true)
 |-- 계약년월: string (nullable = true)
 |-- 거래금액(만원): string (nullable = true)
 |-- 층: string (nullable = true)
 |-- 건축년도: string (nullable = true)



## 전체 데이터의 양 측정
* count()
* len(DataFrame.columns) = 컬럼 수 

In [13]:
print(data.count(),len(data.columns))

9591228 7


## 컬럼의 데이터 타입 바꾸기 (거래금액,전용면적) 숫자로 바꾸기

In [125]:
data = data.withColumn("거래금액(만원)",f.col("거래금액(만원)").cast("int"))

In [126]:
data = data.withColumn("전용면적(㎡)",f.col("전용면적(㎡)").cast("float"))

In [24]:
data.printSchema()

root
 |-- 시군구: string (nullable = true)
 |-- 단지명: string (nullable = true)
 |-- 전용면적(㎡): float (nullable = true)
 |-- 계약년월: string (nullable = true)
 |-- 거래금액(만원): integer (nullable = true)
 |-- 층: string (nullable = true)
 |-- 건축년도: string (nullable = true)



In [25]:
data.summary().show()

DataFrame[summary: string, 시군구: string, 단지명: string, 전용면적(㎡): string, 계약년월: string, 거래금액(만원): string, 층: string, 건축년도: string]

## 결측값 확인

In [30]:
data.select(*(f.sum(f.col(c).isNull().cast("int")).alias(c) for c in data.columns)).show()

+------+------+------------+--------+--------------+---+--------+
|시군구|단지명|전용면적(㎡)|계약년월|거래금액(만원)| 층|건축년도|
+------+------+------------+--------+--------------+---+--------+
|     0|     0|           0|       0|             0|  0|       0|
+------+------+------------+--------+--------------+---+--------+



## 데이터프레임에서 이상값 찾기

In [32]:
bounds = data.stat.approxQuantile("거래금액(만원)",[0.27,0.75],0)

In [33]:
# IQR 계산
lower_range = bounds[0] - (1.5 * (bounds[1] -bounds[0]))
upper_range = bounds[1] + (1.5 * (bounds[1] - bounds[0])) 

In [34]:
print(lower_range)
print(upper_range)

-18500.0
62300.0


In [41]:
data.filter((data["거래금액(만원)"] < lower_range) | (data["거래금액(만원)"] > upper_range)).count()

605879

## 데이터프레임에서 이상 값 찾는 함수

In [ ]:
data.select

In [49]:
def out_lier(data):
    for col_name in data.columns:
        if data.select(col_name).dtypes[0][1] in ("int","float","float32","float62","int32","int64","double"):
            bounds = data.stat.approxQuantile(col_name,[0.27,0.75],0)
            # 이상치 범위 계산 
            
            lower_range = bounds[0] - (1.5 * (bounds[1] -bounds[0]))
            upper_range = bounds[1] + (1.5 * (bounds[1] - bounds[0])) 
            print(col_name,lower_range,upper_range)
            # 이상치 필터링 
            outliers = data.filter((data[col_name] < lower_range) | (data[col_name] > upper_range))
            print(f"number of outliers: {outliers.count()}")
    return outliers.show()
            

In [50]:
out_lier(data)

전용면적(㎡) 21.5 121.5


number of outliers: 669923


거래금액(만원) -18500.0 62300.0


number of outliers: 605879
+--------------------------+----------------+------------+--------+--------------+---+--------+
|                    시군구|          단지명|전용면적(㎡)|계약년월|거래금액(만원)| 층|건축년도|
+--------------------------+----------------+------------+--------+--------------+---+--------+
|경기도 고양일산동구 마두동|강촌마을(미도파)|       134.0|  200601|         63000| 14|    1993|
|경기도 고양일산동구 마두동|강촌마을(미도파)|       134.0|  200601|         63000| 10|    1993|
|경기도 고양일산동구 마두동|  강촌마을(영남)|       165.0|  200601|         78000| 18|    1992|
|경기도 고양일산동구 마두동|  강촌마을(우방)|       134.0|  200601|         71300| 10|    1993|
|경기도 고양일산동구 마두동|  강촌마을(우방)|       134.0|  200601|         73000|  5|    1993|
|경기도 고양일산동구 마두동|  강촌마을(우방)|       134.0|  200601|         67000|  7|    1993|
|경기도 고양일산동구 마두동|  강촌마을(우방)|       164.0|  200601|         87500| 17|    1993|
|경기도 고양일산동구 마두동|  강촌마을(우방)|       164.0|  200601|        102750|  5|    1993|
|경기도 고양일산동구 마두동|  강촌마을(한신)|       173.0|  200601|         78500| 17|    1993|
|경기도 고양일산동구

## 레이블 인코딩

* 데이터프레임의 데이터 타입을 검출한 후 string이면 StringIndexer를 사용해 숫자로 변환

In [51]:
from pyspark.ml.feature import StringIndexer
# indexer = StringIndexer(inputCols=["Sex","Embarked"],outputCols=["SexIndex","EmbarkedIndex"])
# df = indexer.fit(data).transform(data)

In [127]:
def string_index(data):
    for col_name in data.columns:
        if data.select(col_name).dtypes[0][1] == "string":
            indexer = StringIndexer(inputCols=[col_name],outputCols=[col_name+"Index"])
            data = indexer.fit(data).transform(data)
    return data

In [69]:
one_hot = string_index(data)

In [70]:
one_hot.show()

23/11/30 15:20:17 WARN DAGScheduler: Broadcasting large task binary with size 1712.4 KiB
+---------------------------------+-----------+------------+--------+--------------+---+--------+-----------+-----------+-------------+-------+-------------+
|                           시군구|     단지명|전용면적(㎡)|계약년월|거래금액(만원)| 층|건축년도|시군구Index|단지명Index|계약년월Index|층Index|건축년도Index|
+---------------------------------+-----------+------------+--------+--------------+---+--------+-----------+-----------+-------------+-------+-------------+
|강원특별자치도 양구군 양구읍 상리|       경림|        59.0|  200509|          5000| 11|    1998|     1969.0|     2139.0|        206.0|   10.0|          1.0|
|           경기도 의정부시 민락동| 한라비발디|        84.0|  200510|         12750| 19|    2003|       95.0|       25.0|        204.0|   18.0|          9.0|
|           경기도 의정부시 신곡동|      신일1|        59.0|  200510|          5750|  3|    1997|       17.0|     6011.0|        204.0|    2.0|          0.0|
|           경기도 의정부시 신곡동|       풍림|        49.0|

## 레이블 인코딩과 one_hot 인코딩 한 번에 하기 
* 데이터프레임에서 데이터 타입이 string인 컬럼 찾기
* StringIndexer를 이용해 레이블 인코딩(문자 → 숫자)
* 숫자로 변경된 컬럼을 OneHotEncoder를 이용해서 OneHotEncoding

In [128]:
def str2index_onehot(data):
    cols = []
    for col_name in data.columns:
        if data.select(col_name).dtypes[0][1] == "string":
            # StringIndexer를 사용하여 문자열을 인덱스로 변환
            indexer = StringIndexer(inputCols=[col_name], outputCols=[col_name + "Index"])
            data = indexer.fit(data).transform(data)

            # OneHotEncoder를 사용하여 인덱스를 원핫 인코딩으로 변환
            encoder = OneHotEncoder(inputCols=[col_name + "Index"], outputCols=[f"{col_name}_Vec"])
            data = encoder.fit(data).transform(data)

            cols.append(col_name)
            cols.append(col_name + "Index")
            cols.append(col_name+ "_Vec")
    return data, cols

# 함수 사용 예시
# data와 cols는 함수에서 반환하는 값을 받습니다.
data2, cols = str2index_onehot(data)

In [132]:
data2.show(vertical=True)

23/11/30 16:01:00 WARN DAGScheduler: Broadcasting large task binary with size 1762.5 KiB
-RECORD 0-------------------------------------------
 시군구         | 강원특별자치도 양구군 양구읍 상리 
 단지명         | 경림                              
 전용면적(㎡)   | 59.4                              
 계약년월       | 200509                            
 거래금액(만원) | 5000                              
 층             | 11                                
 건축년도       | 1998                              
 시군구Index    | 1969.0                            
 시군구_Vec     | (3653,[1969],[1.0])               
 단지명Index    | 2139.0                            
 단지명_Vec     | (34151,[2139],[1.0])              
 계약년월Index  | 206.0                             
 계약년월_Vec   | (206,[],[])                       
 층Index        | 10.0                              
 층_Vec         | (87,[10],[1.0])                   
 건축년도Index  | 1.0                               
 건축년도_Vec   | (60,[1],[1.0])                    
-RECORD 1---------------------

In [114]:
cols

['시군구',
 '시군구Index',
 '시군구_Vec',
 '단지명',
 '단지명Index',
 '단지명_Vec',
 '전용면적(㎡)',
 '전용면적(㎡)Index',
 '전용면적(㎡)_Vec',
 '계약년월',
 '계약년월Index',
 '계약년월_Vec',
 '거래금액(만원)',
 '거래금액(만원)Index',
 '거래금액(만원)_Vec',
 '층',
 '층Index',
 '층_Vec',
 '건축년도',
 '건축년도Index',
 '건축년도_Vec']

In [133]:
data3 = data2.drop('시군구','시군구Index','단지명','단지명Index','계약년월','계약년월Index','층','층Index','건축년도','건축년도_Vec')

In [134]:
data3.show()

23/11/30 16:01:10 WARN DAGScheduler: Broadcasting large task binary with size 1756.5 KiB
+------------+--------------+-------------------+--------------------+-----------------+---------------+-------------+
|전용면적(㎡)|거래금액(만원)|         시군구_Vec|          단지명_Vec|     계약년월_Vec|         층_Vec|건축년도Index|
+------------+--------------+-------------------+--------------------+-----------------+---------------+-------------+
|        59.4|          5000|(3653,[1969],[1.0])|(34151,[2139],[1.0])|      (206,[],[])|(87,[10],[1.0])|          1.0|
|       84.99|         12750|  (3653,[95],[1.0])|  (34151,[25],[1.0])|(206,[204],[1.0])|(87,[18],[1.0])|          9.0|
|      59.878|          5750|  (3653,[17],[1.0])|(34151,[6011],[1.0])|(206,[204],[1.0])| (87,[2],[1.0])|          0.0|
|       49.83|          5500|  (3653,[17],[1.0])|  (34151,[36],[1.0])|(206,[204],[1.0])| (87,[7],[1.0])|          1.0|
|      129.73|         12200| (3653,[187],[1.0])|(34151,[2206],[1.0])|(206,[205],[1.0])|(87,[10],[1.0])|

In [135]:
data3.printSchema()

root
 |-- 전용면적(㎡): float (nullable = true)
 |-- 거래금액(만원): integer (nullable = true)
 |-- 시군구_Vec: vector (nullable = true)
 |-- 단지명_Vec: vector (nullable = true)
 |-- 계약년월_Vec: vector (nullable = true)
 |-- 층_Vec: vector (nullable = true)
 |-- 건축년도Index: double (nullable = false)



## 독립변수를 1개의 벡터로 변환하고 독립변수와 종속변수만 남기기

In [136]:
from pyspark.ml.feature import VectorAssembler
va = VectorAssembler(inputCols=['전용면적(㎡)','시군구_Vec','계약년월_Vec','층_Vec','건축년도Index'],outputCol='Features')
data3 = va.transform(data3)

data3.show()

23/11/30 16:03:09 WARN DAGScheduler: Broadcasting large task binary with size 1777.6 KiB
+------------+--------------+-------------------+--------------------+-----------------+---------------+-------------+--------------------+
|전용면적(㎡)|거래금액(만원)|         시군구_Vec|          단지명_Vec|     계약년월_Vec|         층_Vec|건축년도Index|            Features|
+------------+--------------+-------------------+--------------------+-----------------+---------------+-------------+--------------------+
|        59.4|          5000|(3653,[1969],[1.0])|(34151,[2139],[1.0])|      (206,[],[])|(87,[10],[1.0])|          1.0|(3948,[0,1970,387...|
|       84.99|         12750|  (3653,[95],[1.0])|  (34151,[25],[1.0])|(206,[204],[1.0])|(87,[18],[1.0])|          9.0|(3948,[0,96,3858,...|
|      59.878|          5750|  (3653,[17],[1.0])|(34151,[6011],[1.0])|(206,[204],[1.0])| (87,[2],[1.0])|          0.0|(3948,[0,18,3858,...|
|       49.83|          5500|  (3653,[17],[1.0])|  (34151,[36],[1.0])|(206,[204],[1.0])| (87,[7],

In [143]:
data3 = data3.withColumnRenamed("거래금액(만원)","price")

In [144]:
data3.columns

['전용면적(㎡)',
 'price',
 '시군구_Vec',
 '단지명_Vec',
 '계약년월_Vec',
 '층_Vec',
 '건축년도Index',
 'Features']

In [147]:
data3 = data3.drop('전용면적(㎡)','시군구_Vec',
 '단지명_Vec',
 '계약년월_Vec',
 '층_Vec',
 '건축년도Index')

In [149]:
data3.show()

+-----+--------------------+
|price|            Features|
+-----+--------------------+
| 5000|(3948,[0,1970,387...|
|12750|(3948,[0,96,3858,...|
| 5750|(3948,[0,18,3858,...|
| 5500|(3948,[0,18,3858,...|
|12200|(3948,[0,188,3859...|
|11500|(3948,[0,188,3859...|
| 4200|(3948,[0,1198,384...|
| 7500|(3948,[0,364,3845...|
| 8500|(3948,[0,364,3845...|
|12500|(3948,[0,364,3845...|
| 5998|(3948,[0,364,3845...|
| 7379|(3948,[0,364,3845...|
| 5580|(3948,[0,364,3845...|
| 7213|(3948,[0,364,3845...|
| 7379|(3948,[0,364,3845...|
| 7379|(3948,[0,364,3845...|
| 7213|(3948,[0,364,3845...|
| 7379|(3948,[0,364,3845...|
| 7379|(3948,[0,364,3845...|
| 8600|(3948,[0,364,3845...|
+-----+--------------------+
only showing top 20 rows



## 홀드아웃

In [151]:
(train_data,test_data) = data3.randomSplit([0.6,0.4],seed=7)

## 회귀분석으로 분석하기

In [152]:
lrm2 = LinearRegression(featuresCol="Features" , labelCol="price")
lrm2 = lrm2.fit(train_data) 

# coefficient
print(lrm2.coefficients)
print(lrm2.intercept)
# prediction 
lrm2_pred = lrm2.evaluate(test_data)

23/11/30 16:11:13 WARN Instrumentation: [92175f78] regParam is zero, which might cause numerical instability and overfitting.


23/11/30 16:11:57 WARN Instrumentation: [92175f78] Cholesky solver failed due to singular covariance matrix. Retrying with Quasi-Newton solver.


[369.95857809144803,17186.920792838835,6790.263186690753,15475.32732131433,15667.243765791622,8727.934915476775,2700.011502706395,15051.313063868902,5509.427666411092,181.67856466035667,12588.906883543428,22775.628701872774,16885.928004587076,-5180.538939238455,11124.025402630434,3015.3842074972194,18384.590601239557,17521.318415103953,4824.8094489240675,3565.9599140680166,38569.9837843777,9783.938255092002,13145.028504721913,4647.629337362515,19219.976674560316,18216.685624124584,22558.085608832407,20443.63601985776,-5994.039180604767,5056.402229604245,20408.41493989002,23097.717997447762,8704.042354792378,-1422.9201984851297,13353.62664443901,4662.174392729346,455.7400179275492,11123.859459165686,26337.834706881575,18242.495529969958,4759.384056158252,16484.699558082688,-3889.52575722087,6807.021515500167,23023.665882706773,1936.038827612829,20591.63350389603,6545.866443542888,1975.8807173510759,2765.3904264363923,3962.455552813143,12860.358562935562,126.84201237209612,2228.802388386

In [159]:
print("MAE" ,lrm2_pred.meanAbsoluteError)
print("MSE", lrm2_pred.meanSquaredError)
print("R2square",lrm2_pred.r2)

MAE 7098.736377942542
MSE 158016531.89785177
R2square 0.7600093622537104


In [156]:
lrm2_pred.predictions.show()

+-----+--------------------+-------------------+
|price|            Features|         prediction|
+-----+--------------------+-------------------+
|  350|(3948,[0,2719,372...| -4437.692583541269|
|  350|(3948,[0,2719,380...| -6509.417153333023|
|  400|(3948,[0,2719,384...| -5733.976564776443|
|  400|(3948,[0,2970,369...|-4013.0069895955385|
|  400|(3948,[0,2970,369...|-4093.7531047250377|
|  400|(3948,[0,3529,376...| -5688.017067756329|
|  475|(3948,[0,2584,377...|-5420.7330094565405|
|  500|(3948,[0,81,3748,...|  90458.57759309892|
|  500|(3948,[0,350,3802...| 13151.246928044828|
|  500|(3948,[0,350,3802...| 13364.565338925226|
|  500|(3948,[0,350,3802...| 14862.310195994563|
|  500|(3948,[0,1983,365...|   4929.26265570696|
|  500|(3948,[0,1983,365...|  9966.678487995698|
|  500|(3948,[0,2382,379...|  3183.314864574466|
|  500|(3948,[0,2936,377...|-3183.1108724874794|
|  500|(3948,[0,2936,379...| -4820.348523548164|
|  525|(3948,[0,2936,381...| -5167.034153062385|
|  550|(3948,[0,1063

In [160]:
result = lrm2_pred.predictions.toPandas()

In [161]:
result

,price,Features,prediction
0,350,"(34.459999084472656, 0.0, 0.0, 0.0, 0.0, 0.0, ...",-4437.692584
1,350,"(34.459999084472656, 0.0, 0.0, 0.0, 0.0, 0.0, ...",-6509.417153
2,400,"(34.4900016784668, 0.0, 0.0, 0.0, 0.0, 0.0, 0....",-5733.976565
3,400,"(44.0099983215332, 0.0, 0.0, 0.0, 0.0, 0.0, 0....",-4013.006990
4,400,"(44.0099983215332, 0.0, 0.0, 0.0, 0.0, 0.0, 0....",-4093.753105
...,...,...,...
3838803,900000,"(217.86000061035156, 0.0, 0.0, 0.0, 0.0, 0.0, ...",176850.133063
3838804,970000,"(228.33999633789062, 0.0, 0.0, 0.0, 0.0, 0.0, ...",224452.513438
3838805,990000,"(198.218994140625, 0.0, 0.0, 0.0, 0.0, 0.0, 0....",178061.393807
3838806,1000000,"(268.6700134277344, 0.0, 0.0, 0.0, 0.0, 0.0, 0...",240034.729580


## 혼자서한 분석

In [71]:
one_hot.columns

['시군구',
 '단지명',
 '전용면적(㎡)',
 '계약년월',
 '거래금액(만원)',
 '층',
 '건축년도',
 '시군구Index',
 '단지명Index',
 '계약년월Index',
 '층Index',
 '건축년도Index']

In [72]:
from pyspark.ml.feature import OneHotEncoder
encoder = OneHotEncoder(inputCols=['시군구Index','단지명Index','계약년월Index','층Index','건축년도Index'],outputCols=['시군구_Vec','단지명_Vec','계약년월_Vec','층_Vec','건축년도_Vec'])
df = encoder.fit(one_hot).transform(one_hot)

In [74]:
df.show()

23/11/30 15:23:31 WARN DAGScheduler: Broadcasting large task binary with size 1736.0 KiB
+---------------------------------+-----------+------------+--------+--------------+---+--------+-----------+-----------+-------------+-------+-------------+-------------------+--------------------+-----------------+---------------+---------------+
|                           시군구|     단지명|전용면적(㎡)|계약년월|거래금액(만원)| 층|건축년도|시군구Index|단지명Index|계약년월Index|층Index|건축년도Index|         시군구_Vec|          단지명_Vec|     계약년월_Vec|         층_Vec|   건축년도_Vec|
+---------------------------------+-----------+------------+--------+--------------+---+--------+-----------+-----------+-------------+-------+-------------+-------------------+--------------------+-----------------+---------------+---------------+
|강원특별자치도 양구군 양구읍 상리|       경림|        59.0|  200509|          5000| 11|    1998|     1969.0|     2139.0|        206.0|   10.0|          1.0|(3653,[1969],[1.0])|(34151,[2139],[1.0])|      (206,[],[])|(87,[10],[1.0])| (60,

In [78]:
df = df.drop('시군구','단지명','계약년월','층','건축년도','시군구Index','단지명Index','계약년월Index','층Index','건축년도Index')

In [80]:
df.printSchema()

root
 |-- 전용면적(㎡): float (nullable = true)
 |-- 거래금액(만원): integer (nullable = true)
 |-- 시군구_Vec: vector (nullable = true)
 |-- 단지명_Vec: vector (nullable = true)
 |-- 계약년월_Vec: vector (nullable = true)
 |-- 층_Vec: vector (nullable = true)
 |-- 건축년도_Vec: vector (nullable = true)



In [81]:
df.columns

['전용면적(㎡)', '거래금액(만원)', '시군구_Vec', '단지명_Vec', '계약년월_Vec', '층_Vec', '건축년도_Vec']

In [82]:
from pyspark.ml.feature import VectorAssembler
feature_assembler = VectorAssembler(inputCols=['전용면적(㎡)','시군구_Vec', '단지명_Vec', '계약년월_Vec', '층_Vec', '건축년도_Vec'],outputCol="Features")
final_df =feature_assembler.transform(df) 
final_df.show()

23/11/30 15:26:32 WARN DAGScheduler: Broadcasting large task binary with size 1772.0 KiB
+------------+--------------+-------------------+--------------------+-----------------+---------------+---------------+--------------------+
|전용면적(㎡)|거래금액(만원)|         시군구_Vec|          단지명_Vec|     계약년월_Vec|         층_Vec|   건축년도_Vec|            Features|
+------------+--------------+-------------------+--------------------+-----------------+---------------+---------------+--------------------+
|        59.0|          5000|(3653,[1969],[1.0])|(34151,[2139],[1.0])|      (206,[],[])|(87,[10],[1.0])| (60,[1],[1.0])|(38158,[0,1970,57...|
|        84.0|         12750|  (3653,[95],[1.0])|  (34151,[25],[1.0])|(206,[204],[1.0])|(87,[18],[1.0])| (60,[9],[1.0])|(38158,[0,96,3679...|
|        59.0|          5750|  (3653,[17],[1.0])|(34151,[6011],[1.0])|(206,[204],[1.0])| (87,[2],[1.0])| (60,[0],[1.0])|(38158,[0,18,9665...|
|        49.0|          5500|  (3653,[17],[1.0])|  (34151,[36],[1.0])|(206,[204],[1.0

In [85]:
final_df = final_df.drop('전용면적(㎡)','시군구_Vec', '단지명_Vec', '계약년월_Vec', '층_Vec', '건축년도_Vec')

In [101]:
final_df.show()

23/11/30 15:34:32 WARN DAGScheduler: Broadcasting large task binary with size 1741.9 KiB
+--------------+--------------------+
|거래금액(만원)|            Features|
+--------------+--------------------+
|          5000|(38158,[0,1970,57...|
|         12750|(38158,[0,96,3679...|
|          5750|(38158,[0,18,9665...|
|          5500|(38158,[0,18,3690...|
|         12200|(38158,[0,188,586...|
|         11500|(38158,[0,188,586...|
|          4200|(38158,[0,1198,65...|
|          7500|(38158,[0,364,517...|
|          8500|(38158,[0,364,517...|
|         12500|(38158,[0,364,517...|
|          5998|(38158,[0,364,517...|
|          7379|(38158,[0,364,517...|
|          5580|(38158,[0,364,517...|
|          7213|(38158,[0,364,517...|
|          7379|(38158,[0,364,517...|
|          7379|(38158,[0,364,517...|
|          7213|(38158,[0,364,517...|
|          7379|(38158,[0,364,517...|
|          7379|(38158,[0,364,517...|
|          8600|(38158,[0,364,517...|
+--------------+--------------------+
only 

In [97]:
(train_data,test_data) = final_df.randomSplit([0.6,0.4],seed=10)

In [98]:
from sklearn.linear_model import LinearRegression

In [102]:
from pyspark.ml.regression import LinearRegression
lrm = LinearRegression(featuresCol="Features", labelCol="거래금액(만원)")
lrm = lrm.fit(train_data)
lrm_pred = lrm.transform(test_data)
lrm_pred.show()

23/11/30 15:35:28 WARN DAGScheduler: Broadcasting large task binary with size 4.6 MiB


23/11/30 15:36:11 WARN DAGScheduler: Broadcasting large task binary with size 4.6 MiB


23/11/30 15:36:12 WARN DAGScheduler: Broadcasting large task binary with size 4.6 MiB


23/11/30 15:36:55 WARN DAGScheduler: Broadcasting large task binary with size 4.6 MiB
23/11/30 15:36:55 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
23/11/30 15:36:55 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


23/11/30 15:36:56 WARN DAGScheduler: Broadcasting large task binary with size 4.6 MiB


...

23/11/30 15:37:02 WARN DAGScheduler: Broadcasting large task binary with size 4.6 MiB


23/11/30 15:37:03 WARN DAGScheduler: Broadcasting large task binary with size 4.6 MiB
23/11/30 15:37:03 WARN DAGScheduler: Broadcasting large task binary with size 4.6 MiB
23/11/30 15:37:03 WARN DAGScheduler: Broadcasting large task binary with size 4.6 MiB
23/11/30 15:37:04 WARN DAGScheduler: Broadcasting large task binary with size 4.6 MiB
23/11/30 15:37:04 WARN DAGScheduler: Broadcasting large task binary with size 4.6 MiB
23/11/30 15:37:04 WARN DAGScheduler: Broadcasting large task binary with size 4.6 MiB
23/11/30 15:37:05 WARN DAGScheduler: Broadcasting large task binary with size 4.6 MiB
23/11/30 15:37:05 WARN DAGScheduler: Broadcasting large task binary with size 4.6 MiB
23/11/30 15:37:05 WARN DAGScheduler: Broadcasting large task binary with size 4.6 MiB
23/11/30 15:37:05 WARN DAGScheduler: Broadcasting large task binary with size 4.6 MiB
23/11/30 15:37:06 WARN DAGScheduler: Broadcasting large task binary with size 4.6 MiB
23/11/30 15:37:06 WARN DAGScheduler: Broadcasting larg

23/11/30 15:38:36 WARN DAGScheduler: Broadcasting large task binary with size 4.9 MiB


23/11/30 15:38:36 WARN DAGScheduler: Broadcasting large task binary with size 4.9 MiB


+--------------+--------------------+-------------------+
|거래금액(만원)|            Features|         prediction|
+--------------+--------------------+-------------------+
|           300|(38158,[0,2719,60...| -6054.419482411144|
|           350|(38158,[0,2719,60...| -5265.223193744128|
|           400|(38158,[0,2719,60...| -6112.746924267241|
|           400|(38158,[0,2970,18...| -2969.701294544844|
|           400|(38158,[0,2970,18...| -2859.499930259256|
|           400|(38158,[0,3529,32...| -4520.104168246416|
|           475|(38158,[0,2584,15...|-3749.8395664017035|
|           500|(38158,[0,350,482...| 12010.186251796866|
|           500|(38158,[0,350,482...|  12032.77404386088|
|           500|(38158,[0,350,482...| 12399.761108115696|
|           500|(38158,[0,1983,77...|  10821.89716452942|
|           500|(38158,[0,2382,36...| 2868.3446563533116|
|           500|(38158,[0,2936,15...|-4705.5232096228865|
|           513|(38158,[0,2936,15...| -5389.514853604227|
|           513|(381